## Install and load Ariadne extension

In [1]:
!pip install -I git+https://github.com/alanbraz/ipython-ariadne-extension.git

  Cloning https://github.com/alanbraz/ipython-ariadne-extension.git to /home/dsxuser/.tmp/pip-37_kcgl7-build
  Running setup.py install for ipython-ariadne-extension ... done


In [2]:
%load_ext ariadne

In [3]:
1+1

2

Adriane check OK!


## Load MNIST data

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Adriane check OK!


## Define TensorFlow estimator

In [5]:
import tensorflow as tf

def model_fn(features, labels, mode):
    num_classes = 10
    dropout = 0.75

    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)

    pred_classes = tf.argmax(logits_test, axis=1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)

    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    return tf.estimator.EstimatorSpec(
        mode=mode, predictions=pred_classes, loss=loss_op, train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

Adriane check OK!


## Define conv_net

In [6]:
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    xyz = x_dict['images']

    bad_z = tf.reshape(xyz, shape=[-1, 11, 28, 1])
    z = tf.reshape(xyz, shape=[-1, 28, 28, 1])

    bad_conv1 = tf.layers.conv2d(xyz, 32, 5, activation=tf.nn.relu)
    conv1 = tf.layers.conv2d(z, 32, 5, activation=tf.nn.relu)
        
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
    conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
    fc1 = tf.contrib.layers.flatten(conv2)
    fc1 = tf.layers.dense(fc1, 1024)
    fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
    return tf.layers.dense(fc1, n_classes)

def apply(model=None, action=None):
    if model is None:
        model = tf.estimator.Estimator(model_fn)
    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'images': mnist.train.images}, y=mnist.train.labels,
        batch_size=128, num_epochs=None, shuffle=True)
    if action == 'train':
        model.train(input_fn, steps=2000)
    return model

model = apply(action='configure')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/dsxuser/.tmp/tmpvjqnjala', '_session_config': None, '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_save_checkpoints_secs': 600}


Adriane Diagnostic Warning: Cannot reshape pixel[n][28 * 28] to pixel[?][11][28][1]
Cell [6] Line 4: bad_z = tf.reshape(xyz, shape=[-1, 11, 28, 1])
                                    ^^^
Adriane Diagnostic Error: Bad type to convolve pixel[n][28 * 28], needs 4 dimensions (possible fix: tf.reshape(xyz, [-1, 28, 28, 1]))
Cell [6] Line 7: bad_conv1 = tf.layers.conv2d(xyz, 32, 5, activation=tf.nn.relu)
                                              ^^^


## Train with TensorFlow

In [7]:
apply(model, action='train')

ValueError: Dimension size must be evenly divisible by 308 but is 100352 for 'Reshape' (op: 'Reshape') with input shapes: [128,784], [4] and with input tensors computed as partial shapes: input[1] = [?,11,28,1].

Adriane check OK!
